# Cohort Analysis of Opinions on LGBT issues

For this project, we will be analyzing at data from the General Social Survey (GSS) to explore the opinions of different demographics on LGBT issues.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import thinkstats2
import thinkplot

In [3]:
def read_gss(dirname):
    """Reads GSS files from the given directory.
    
    dirname: string
    
    returns: DataFrame
    """
    dct = utils.read_stata_dict(dirname + '/GSS.dct')
    gss = dct.read_fixed_width(dirname + '/GSS.dat.gz',
                             compression='gzip')
    return gss

In [ ]:
# Read in the GSS variables.
read_gss('gss_vars')

## Clean the Data